# **就是说记录一下如何使用PaddleYOLO这个仓库**

## **0. 数据集对齐PaddleYOLO**

```python
# my voc-OD-dataset structure
VOC_2007:
~/VOCDevkit/VOC2007
    - /Annotations   # *.xml
    - /ImageSets
        - /Main      # *.txt
    - /JPEGImages    # *.jpg

# target voc-OD-dataset structure
paddledet_voc:
~/dataset/paddledet_voc
    - label_list.txt
    - trainval.txt   :path/to/(*.jpg *.xml) VOCDevkit/VOC2007/*.jpg VOCDevkit/VOC2007/
    - test.txt       :path/to/(*.jpg *.xml) VOCDevkit/VOC2007/*.jpg VOCDevkit/VOC2007/
    - train.txt      :path/to/(*.jpg *.xml) VOCDevkit/VOC2007/*.jpg VOCDevkit/VOC2007/
    - val.txt        :path/to/(*.jpg *.xml) VOCDevkit/VOC2007/*.jpg VOCDevkit/VOC2007/
    - /VOCDevkit/2007
        - /Annotations  # *.xml   
        - /ImageSets
            - /Main     # *.txt  # line:path/to/data/*  
        - /JPEGImages   # *.jpg
```

In [2]:
import os, shutil

paddledet_vocdataset_root = r"dataset/paddledet_voc"
os.makedirs(paddledet_vocdataset_root, exist_ok=True)

def copy_dirs(src_dir, dst_dir, move=False):
    for root, dirs, files in os.walk(src_dir):
        # 计算相对路径
        relative_path = os.path.relpath(root, src_dir)
        # 创建目标文件夹中的子文件夹
        dst_subdir = os.path.join(dst_dir, relative_path)
        if not os.path.exists(dst_subdir):
            os.makedirs(dst_subdir)
        
        # 复制文件
        for file in files:
            src_file = os.path.join(root, file)
            dst_file = os.path.join(dst_subdir, file)
            if move:
                shutil.move(src_file, dst_file)
            else:
                shutil.copyfile(src_file, dst_file)


target_dir  =r"PaddleYOLO/dataset/paddledet_voc"
os.makedirs(target_dir, exist_ok=True)
copy_dirs(paddledet_vocdataset_root, target_dir)

In [3]:
# rename  my_classes.txt to label_list.txt
os.rename(os.path.join(target_dir, "my_classes.txt"), os.path.join(target_dir, "label_list.txt"))

In [5]:

# 用于训练的数据集root: target_dir  =r"PaddleYOLO/dataset/paddledet_voc"
# JPEGImages: 图片文件夹  os.path.join(target_dir, "VOCDevkit/VOC2007/JPEGImages")
# Annotations: 标注文件夹 os.path.join(target_dir, "VOCDevkit/VOC2007/Annotations")

def get_paddetVocTxt(dataset_root="PaddleYOLO/dataset/paddledet_voc", txt_name="trainval.txt"):
    txtpth = os.path.join(os.path.join(dataset_root, "VOCDevkit/VOC2007/ImageSets/Main"), txt_name)
    with open(txtpth, 'r', encoding='utf-8') as f:
        lines = f.readlines()    
    lists_path = os.path.join(dataset_root, txt_name)
    lists = open(lists_path, 'w', encoding='utf-8')
    
    
    for line in lines:
        image_line = os.path.normpath(os.path.join("VOCDevkit/VOC2007/JPEGImages", line.strip()))
        anno_line = os.path.normpath(os.path.join("VOCDevkit/VOC2007/Annotations", line.strip()))
        lists_line = f"{image_line}.jpg  {anno_line}.xml\n"
        lists.write(lists_line)
    lists.close()

get_paddetVocTxt(txt_name="trainval.txt")
get_paddetVocTxt(txt_name="train.txt")
get_paddetVocTxt(txt_name="val.txt")
get_paddetVocTxt(txt_name="test.txt")

**就是简单记录一下(现在可能用不上)**<br>
从图片文件集images、标注文件集annotaions  来构建我们的数据集train_set val_set test_set

```python
~/dataset/
    - annotaions      # 存放标注文件,如 *.xml
    - images          # 存放图像文件,如 *.jpg
    - trainval_test_split/
        - split_log   # trainval.txt  train.txt val.txt test.txt
    - train_set/
        - annotaions  # 存放标注文件,如 *.xml
        - images      # 存放图像文件,如 *.jpg
    - val_set/
        - annotaions  # 存放标注文件,如 *.xml
        - images      # 存放图像文件,如 *.jpg
    - test_set/
        - annotaions  # 存放标注文件,如 *.xml
        - images      # 存放图像文件,如 *.jpg
    - trainval_trainlines.txt  # path/to/data/* annotations_1 annotations_2 ...
    - trainval_vallines.txt
```

```python

import os
import random
import shutil
from tqdm import tqdm 
import xml.etree.ElementTree as ET
import numpy as np
from utils.utils import get_classes

classes_path = r"model_data/my_classes.txt"
trainval_percent = 0.9
train_percent = 0.9
dataset_root = "dataset"
dataset_structure_original = {'image': 'images', 'label': 'annotations'}

# -> split_log : trainval.txt train.txt val.txt  test.txt
split_log_dir = r"dataset/split_log"
os.makedirs(split_log_dir, exist_ok=True)

dataset_structure_target = [{'sets': {
                              'sets': ['train', 'val', 'test'],
                              'train': 'dataset/train_set',
                              'val': 'dataset/val_set',
                              'test': 'dataset/test_set'}},
                            {'sets_stru': {'image': 'images', 'label': 'annotations'}}]

classes_list, classes_num = get_classes(classes_path)
print(len(dataset_structure_target))


# split data
print("Generate txt in splir_log_dir ...")

xmlfilepath = os.path.join(dataset_root, dataset_structure_original['label'])
xmlfiles = os.listdir(xmlfilepath)
total_xml = []

for xml in xmlfiles:
    if xml.endswith('.xml'):
        total_xml.append(xml)

nums = len(total_xml)
lists = range(nums)
trainval_nums = int(nums * trainval_percent)
train_nums = int(trainval_nums * train_percent)
trainval = random.sample(lists, trainval_nums)
train = random.sample(trainval, train_nums)
print("train and val size", trainval_nums)
print("train size", train_nums)

ftrainval = open(os.path.join(split_log_dir, 'trainval.txt'), 'w', encoding='utf-8')
ftest = open(os.path.join(split_log_dir, 'test.txt'), 'w', encoding='utf-8')
ftrain = open(os.path.join(split_log_dir, 'train.txt'), 'w', encoding='utf-8')
fval = open(os.path.join(split_log_dir, 'val.txt'), 'w', encoding='utf-8')

for i in lists:
    name = total_xml[i][:-4]+'\n'  # 仅保留文件名
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)

ftrainval.close()
ftrain.close()
fval.close()
ftest.close()
print("Generate txt in splir_log_dir done.")


# 移动数据集
def get_target_dataset(dataroot = dataset_root,
                       log_dir = split_log_dir,
                       train_slice='train.txt', 
                       val_slice='val.txt', 
                       tes_slice='test.txt', 
                       target_dataset_stru=dataset_structure_target):
    
    with open(os.path.join(log_dir, train_slice), 'r', encoding='utf-8') as f:
        trains = f.readlines()
    with open(os.path.join(log_dir, val_slice), 'r', encoding='utf-8') as f:
        vals = f.readlines()
    with open(os.path.join(log_dir, tes_slice), 'r', encoding='utf-8') as f:
        tests = f.readlines()
    
    trainset_root = target_dataset_stru[0]['sets']['train']
    valset_root = target_dataset_stru[0]['sets']['val']
    testset_root = target_dataset_stru[0]['sets']['test']
    os.makedirs(trainset_root, exist_ok=True)
    os.makedirs(valset_root, exist_ok=True)
    os.makedirs(testset_root, exist_ok=True)
    
    def move_datas(lines, data_root, save_root, stru=dataset_structure_target):
        for li in tqdm(lines):
            li = li.strip()
            # 移动图像文件
            image_dir = os.path.join(data_root, stru[1]['sets_stru']['image'])
            img_path = os.path.join(image_dir, f"{li}.jpg")

            if os.path.exists(img_path):
                image_save_dir = os.path.join(save_root, stru[1]['sets_stru']['image'])
                os.makedirs(image_save_dir, exist_ok=True)
                image_save_path = os.path.join(image_save_dir, f"{li}.jpg")
                shutil.copyfile(img_path, image_save_path)
            else:
                print(f'{li}.jpg图片文件 is not file !')
            
            # 移动标签文件
            label_dir = os.path.join(data_root, stru[1]['sets_stru']['label'])
            xml_path = os.path.join(label_dir, f"{li}.xml")
            
            if os.path.exists(xml_path):
                xml_save_dir = os.path.join(save_root, stru[1]['sets_stru']['label'])
                os.makedirs(xml_save_dir, exist_ok=True)
                xml_save_path = os.path.join(xml_save_dir, f"{li}.xml")
                shutil.copyfile(xml_path, xml_save_path)
            else:
                print(f'{li}.xml标注文件 is not file !')
    
    move_datas(trains, dataroot, trainset_root)
    move_datas(vals, dataroot, valset_root)
    move_datas(tests,dataroot, testset_root)
    
    
# 生成训练用的： trainval_trainlines.txt   trainval_vallines.txt

with open(os.path.join(split_log_dir, 'train.txt'), 'r', encoding='utf-8') as f:
    trains = f.readlines()
with open(os.path.join(split_log_dir, 'val.txt'), 'r', encoding='utf-8') as f:
    vals = f.readlines()

def convert_annotations(image_id, linesfile):
    # 原始数据的文件夹  xml_dir
    infiles = open(os.path.join("dataset/paddledet_voc/VOCDevkit/VOC2007/Annotations", f"{image_id}.xml"), 'r', encoding='utf-8')
    tree = ET.parse(infiles)
    root = tree.getroot()
    
    for obj in root.iter('object'):
        difficult = 0
        if obj.find('difficult') != None:
            difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes_list or int(difficult)==1:
            continue
        cls_id = classes_list.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(float(xmlbox.find('xmin').text)), 
         int(float(xmlbox.find('ymin').text)), 
         int(float(xmlbox.find('xmax').text)), 
         int(float(xmlbox.find('ymax').text)))
        # x1 y1 x2 y2 cls_id
        linesfile.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))
        
        # nums[classes_list.index(cls)] = nums[classes_list.index(cls)] + 1


def get_lines(dataroot="dataset", splitLogtext="dataset/split_log/train.txt", trainvalLinesText="trainval_trainlines.txt"):
    # train.txt or val.txt
    
    image_ids = open(splitLogtext, 'r', encoding='utf-8').read().strip().split()
    listfiles = open(trainvalLinesText, 'w', encoding='utf-8')
    
    image_dir = os.path.abspath(os.path.join(dataroot, 'images'))
    for image_id in image_ids:
        image_path = os.path.join(image_dir, f"{image_id}.jpg")
        listfiles.write(image_path)
        convert_annotations(image_id, listfiles)
        listfiles.write('\n')
    listfiles.close()
    
    print(f"Generate {trainvalLinesText} for train done.")
    
    
# trainval_trainlines.txt  
get_lines(dataroot="dataset", splitLogtext="dataset/split_log/train.txt",trainvalLinesText="trainval_trainlines.txt")


# trainval_vallines.txt
get_lines(dataroot="dataset", splitLogtext="dataset/split_log/val.txt",trainvalLinesText="trainval_vallines.txt")
```

# 1. PaddleYOLO 仓库结构分析

```python
# PaddleYOLO
~/PaddleYOLO
    - /configs    # 各yolo的配置入口 
    - /dataset    # 数据集入口，有提供默认数据集的下载、处理脚本
    - /demo       # 几张用于测试的图片
    - /deploy     # 部署入口
    - /docs       # 工程文档入口，教程都在这里
    - /ppdet      # import ppdet  # ppdet package源码 (setup.py中有package的构建信息)
    - /scripts    # 
    - /test_tipc  # paddle包装的训练测试开发工具入口
    - /tools      # 集成的脚本  如train.py   infer.py  ...
    - requirements.txt 、setup.py  # 用于配置/构建PaddleYOLO工程环境
    - README.md   # 工程README.md，优先看
```



# 2. 配置对齐

PaddleYOLO这样的仓库是有学习成本的，但使用逻辑相似。

**！！！记住先看工程README.MD ！！！**

- 用起来只需要：
    - 对齐数据集    --> 仓库的要求格式 
    - 构建配置文件  --> 可能需要构建多个.yaml  .yml文件

## 2.1 数据集配置  模型配置 优化器配置 训练配置 验证配置 测试配置  ...

PaddleYOLO/dataset/configs/datasets/paddledet_voc.yml or .yaml

**需要注意一下， yaml文件中就把所有的中文字符删除**

```yaml
metric: VOC # 目前支持COCO, VOC, WiderFace等评估标准
map_type: 11point
num_classes: 3 # 数据集的类别数，不包含背景类，roadsign数据集为4类，其他数据需要修改为自己的数据类别

TrainDataset:
  !VOCDataSet
    dataset_dir: dataset/paddledet_voc # 训练集的图片所在文件相对于dataset_dir的路径
    anno_path: train.txt # 训练集的标注文件相对于dataset_dir的路径
    label_list: label_list.txt # 数据集所在路径，相对于PaddleDetection路径
    data_fields: ['image', 'gt_bbox', 'gt_class', 'difficult'] # 控制dataset输出的sample所包含的字段，注意此为训练集Reader独有的且必须配置的字段

EvalDataset:
  !VOCDataSet
    dataset_dir: dataset/paddledet_voc # 数据集所在路径，相对于PaddleDetection路径
    anno_path: val.txt # 验证集的标注文件相对于dataset_dir的路径
    label_list: label_list.txt # 标签文件，相对于dataset_dir的路径
    data_fields: ['image', 'gt_bbox', 'gt_class', 'difficult']

TestDataset:
  !ImageFolder
    anno_path: label_list.txt # 标注文件所在路径，仅用于读取数据集的类别信息，支持json和txt格式
    dataset_dir: dataset/paddledet_voc 
```


新建与训练权重的文件夹,将preweights文件夹移过来<br>
PaddleYOLO/configs/yolov8/preweights  

配置工程主配置文件
```yaml

_BASE_: [
  '../datasets/p.yml', # 指定为自定义数据集配置路径
  '../runtime.yml',
  '_base_/optimizer_40e.yml',
  '_base_/yolov3_mobilenet_v1.yml',
  '_base_/yolov3_reader.yml',
]
pretrain_weights: configs/yolov8/preweights/yolov8_s_500e_coco.pdparams
weights: output/yolov8_s_pdetVOC/model_final

YOLOv3Loss:
  ignore_thresh: 0.7
  label_smooth: true

```


```terminal
set CUDA_VISIBLE_DEVICES=0 

python tools/train.py -c configs/yolov8/yolov8_s_pdetvoc.yml --eval
```